<a href="https://colab.research.google.com/github/SophiaLeiker/JCR_StratificationSampling/blob/main/Sophia's_Copy_of_Jamala_Canyon_Stratification_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stratification & Sampling Design for Jalama Canyon Ranch

*Monitor: Sam Bennetts* 

*Last Edited: July 2, 2021*

*Last Used: July 2, 2021*

---


This notebook outlines the methods used to determine soil sample locations for the *White Buffalo Land trust* rotational grazing project at the Jamala Canyon ranch in Central California. To ensure the sampling design captured soil carbon variability, a *k-means* clustering algorithm was used to stratify the project area into subregions, or strata, which shared similar soil characteristics. Sample locations were then selected using a conditioned latin hypercube (cLHS) algorithm to randomly select sampling locations within each of the strata.

## Notebook Setup

In [ ]:
%%capture
!pip uninstall python_dateutil numpy -y
!pip install python-dateutil==2.7.5
!pip install numpy==1.20.1
!pip install git+https://gitlab.com/S4mmyB/stratipy.git@master
!pip install git+https://github.com/creare-com/pydem.git@develop # import develop branch is python3
!pip install clhs folium geopandas
!pip install aiohttp
!pip install xmltodict

In [ ]:
import os
from google.colab import drive

# mount to google drive
drive.mount('/content/drive')

# set working directories
working_dir = r'/content/drive/Shared drives/Regen - Science/Grasslands/Grasslands Projects/White Buffalo Land Trust/Jamala Canyon'
data_dir = os.path.join(working_dir, 'data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Property Overview

The Pilango Project is located in Wallace County, Kansas in the temperate indigenous grasslands of the Great Plains of the U.S. The project boundaries provided by the land manager are shown below. 

In [ ]:
import folium
import geopandas as gpd
import shapely
from stratipy import utils

# load in project boundary
field_boundary_geojson = os.path.join(data_dir, 'jamala_grassland_area.geojson')
field_boundary = gpd.read_file(field_boundary_geojson)

# pull in field centroid to center folium plot
field_centroid = field_boundary.geometry[0].centroid
field_center = [field_centroid.y, field_centroid.x]

# define function to generate a folium map object
def get_basemap(map_center=field_center, zoom_start=15):
    return folium.Map(location=map_center, 
               zoom_start=zoom_start, 
               tiles='https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.png?access_token=pk.eyJ1Ijoib3Vyc2NpIiwiYSI6ImNqb2ljdHMxYjA1bDAzcW03Zjd0cHBsbXMifQ.rL9QPLvi0kLP3DzLt1PQBA',
               attr='map copyright Mapbox',
    )

basemap = get_basemap()
boundary_layer = folium.GeoJson(data=field_boundary, 
                                style_function=lambda x: {'fillColor': '#00000000'},
                                name='Project Boundary').add_to(basemap)

basemap

ContextualVersionConflict: ignored

In [ ]:
def get_utm_epsg(latitude, longitude):
    """
    gets wgs84 utm zone for coordinates input in epsg:4326
    """
    offset = int(round((183 + latitude) / 6.0))
    return 32600 + offset if longitude > 0 else 32700 + offset

def check_project_bounds(total_bounds):
    """
    total bounds = [llx, lly, urx, ury]
    """
    ll_epsg = get_utm_epsg(total_bounds[0], total_bounds[1])
    ur_epsg = get_utm_epsg(total_bounds[2], total_bounds[3])

    if ll_epsg != ur_epsg:
        print('The geometry crosses utm zones')
        return None
    
    return ll_epsg

# make sure the project falls does not overlap multiple utm zones
proj_utm_epsg = check_project_bounds(field_boundary.total_bounds)

# calculate grassland area
field_boundary_utm = field_boundary.to_crs(epsg=proj_utm_epsg)
grasslands_area_m2 = float(field_boundary_utm.area.sum())
grasslands_area_ha = grasslands_area_m2 / 10000
print(f'The Jamala Canyon ranch grasslands area totals {grasslands_area_ha:.2f} hectares.')

In [ ]:
import shapely.geometry

def get_bbox(total_bounds, padding=0):
    # use boundary extent to create a padded bbox for processing
    return shapely.geometry.box(*total_bounds).buffer(padding, cap_style=2, join_style=2).bounds

bbox = get_bbox(field_boundary.total_bounds, 1e-3)

## Variables Used to Stratify

Variables highly correlated to soil organic carbon can be used as proxies to divide the project area into
strata which encompass the full range of SOC levels (low, medium and high). Variables found to be good proxies to spatial variability of SOC at the field scale include:
* **Topographic**: elevation, slope, aspect, erosion, & terrain ruggedness Index (TRI)
* **Land Use / Land cover (LULC)**: Vegetation cover, above ground biomass, land management history
* **Spectral Indices from Satellite Imagery**: NDVI , BSI, NDWI, Tasseled Cap
* **Hydrologic**: topographic wetness index (TWI), catchment area and stream power index (SPI)
* **Pedologic**: soil type, organic matter, clay content, pH <br />

The variables used for the Pilango project are outlined below. 

### NDVI

NDVI was derived using cloud free Sentinel-2 satellite images taken over the project area between April, 1 2020 and April 1, 2021. The NDVI was calculated for each of the images individually; the image with the highest range NDVI value was selected and used for analysis. 

---

*Images used in this analysis were downloaded the [AWS Open Data Portal](https://registry.opendata.aws/sentinel-2-l2a-cogs/). As the project area is covered by multiple tile footprints which different spatial projections, only images with the Sentinel-2 L1C Tile ID: T14SKJ were used for analysis.*

In [ ]:
from stratipy import sentinel2, utils

# search criteria
start_date = '2020-05-01'
end_date = '2021-05-01'
cloud_cover = 5
catalog = sentinel2.get_s2_l2a_catalog(bbox, start_date, end_date, cloud_cover)

# print images used for the analysis
print("Sentinel-2 Tile IDs for the {} images used for analysis".format(len(catalog)))
print("-"*60)
for k, v in catalog.items():
    print(v.metadata['sentinel:product_id'])

In [ ]:
import asyncio
import nest_asyncio
import numpy as np
nest_asyncio.apply()

summary_method = 'max'
async def ndvi_await():
    return await sentinel2.get_ndvi_summary_by_bbox_async(bbox, catalog, summary_method)

greenest, greenest_tf, greenest_crs = asyncio.run(ndvi_await())

In [ ]:
print(f'The range of NDVI values is {np.nanmin(greenest):.3f} - {np.nanmax(greenest):.3f}')

### Elevation
The elevation variables selected for the analysis are: elevation, slope, aspect, and topographic wetness index. Elevation data was sourced using the [USGS 3DEP National Indexing Scheme](https://www.usgs.gov/core-science-systems/ngp/3dep/3dep-national-indexing-scheme); all other variables were calculated using the original elevation data. 

In [ ]:
%%capture
from stratipy import elevation
from pydem.dem_processing import DEMProcessor

async def elev_await():
  return await elevation.fetch_elevation(
  bbox, 
  dst_shape=greenest.shape,
  dst_transform=greenest_tf,
  dst_crs=greenest_crs,
)

# calculate elevation, slope, aspect
elev, elev_tf, elev_crs = asyncio.run(elev_await())
slope = elevation.calculate_slope(elev)
aspect = elevation.calculate_aspect(elev)

# calculate twi
elev_of = 'elevation.tif'
utils.write_geotiff(elev_of, elev, greenest_tf, greenest_crs)
dem_proc = DEMProcessor(elev_of)
twi = dem_proc.calc_twi()

In [ ]:
print(f'The topographic relief across the area is values is {np.nanmax(elev) - np.nanmin(elev):.3f} feet.')

### Soils
Soil data were downloaded from the [NRCS SSURGO Database](https://datagateway.nrcs.usda.gov/). Organic matter and clay values were derived by averaging their percent content across all horizons, taking into account the horizon depth.  
    

In [ ]:
from stratipy import soils

async def soils_await():
  return await soils.get_clipped_summary(bbox)

soil_results = asyncio.run(soils_await()) 

om, clay, res_tf = soils.get_om_clay_rasters(
    soil_results, 
    bbox, 
    greenest.shape, 
    transform=greenest_tf,
    dst_crs=greenest_crs
)

### Masking Variables w/Project Boundary



In [ ]:
import rasterio as rio

# reproject project boundary to utm
boundary_utm = field_boundary.to_crs(greenest_crs)
boundary_shp = boundary_utm.geometry
boundary_shp_buffered = boundary_shp.buffer(1e-3, cap_style=2, join_style=2)

# create raster mask
raster_mask = rio.features.geometry_mask(
    boundary_shp,
    out_shape=greenest.shape,
    transform=greenest_tf,
    all_touched=True
)

# clip rasters to project boundary
masked_greenest, masked_slope, masked_aspect, masked_twi, masked_clay, masked_om = [
    utils.mask_raster_with_geometry(
        raster,
        greenest_tf,
        boundary_shp,
        greenest_crs
    )[0] 
    for raster in [greenest, slope, aspect, twi, clay, om]
]

### Visualization

In [ ]:
from matplotlib import cm
from sklearn.preprocessing import MinMaxScaler


cmaps = {
    'NDVI': cm.Greens, 
    'Aspect': cm.Spectral, 
    'Slope': cm.Spectral, 
    'TWI': cm.Spectral,
    'Organic Matter': cm.Pastel2, 
    'Clay': cm.Dark2
}

In [ ]:
basemap = get_basemap()
boundary_layer = folium.GeoJson(data=boundary_utm, 
                                style_function=lambda x: {'fillColor': '#00000000'},
                                name='Project Boundary').add_to(basemap)


image_bounds = [[bbox[1], bbox[0]], [bbox[3], bbox[2]]]
raster_layers = [masked_greenest, masked_slope, masked_aspect, masked_twi, masked_clay, masked_om]
layer_names = ['NDVI', 'Slope', 'Aspect', 'TWI', 'Clay' 'Organic Matter']
scaler = MinMaxScaler()
for layer, name in zip(raster_layers, layer_names):
    try:
        layer = scaler.fit_transform(layer)
        basemap.add_child(folium.raster_layers.ImageOverlay(
            layer,
            image_bounds,
            colormap=cmaps[name],
            name=name,
        ))
    except Exception:
        continue

basemap.add_child(folium.LayerControl())
basemap

### Feature Selection
Of the six variables tested in the stratification, only three were selected for analysis. The following variables were excluded: 

*   **Organic Matter**: A comparison between the clay and organic matter layers reveals that clay is much more variable across the project area than organic matter (organic matter only has 4 unique values while clay has 17. Visually comparing the two layers, you can see that the spatial distribution of clay within the project area captures the distibution of OM, thus the organic matter layer was removed. 
 
*   List item



**The variables used for this stratification were**: *NDVI, slope, and organic matter, TWI*.

In [ ]:
import numpy as np
import clhs
import pandas as pd
from sklearn import preprocessing
from scipy.spatial import distance_matrix
import math

del clhs_inputs

clhs_inputs = pd.DataFrame({
    'ndvi': np.nan_to_num(masked_greenest.flatten()),
    'slope': np.nan_to_num(masked_slope.flatten()),
    'aspect': np.nan_to_num(masked_aspect.flatten()),
    'twi': np.nan_to_num(masked_twi.flatten()),
    'clay': np.nan_to_num(masked_clay.flatten()),
})

clhs_inputs = clhs_inputs[
    (clhs_inputs["ndvi"] != 0) & 
    (clhs_inputs["slope"] != 0) &
    (clhs_inputs["aspect"] != 0) &
    (clhs_inputs["twi"] != 0) &
    (clhs_inputs["clay"] != 0) 
]


clhs_inputs_st = preprocessing.scale(clhs_inputs)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

cluster_range = range(1,20)
cluster_range_abbr = range(2,20)
cluster_wss = []

from sklearn.cluster import KMeans
for num_cluster in cluster_range:
    clusters = KMeans(num_cluster)
    clusters.fit(clhs_inputs_st)
    cluster_wss.append(clusters.inertia_)

cluster_range_ind = range(1,len(cluster_range))
cluster_range_ind

cluster_slope = []
for num_cluster in cluster_range_ind:
    cluster_slope.append(cluster_wss[num_cluster]-cluster_wss[num_cluster-1])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

plt.xlabel('# Clusters')
plt.ylabel('WSS')
plt.plot(cluster_range_abbr, cluster_slope, marker = 'o')
for xy in zip(cluster_range_abbr, cluster_slope):                                      
    ax.annotate('%s' % xy[0], xy=xy, textcoords='data', 
                 horizontalalignment='right', verticalalignment='bottom') #
plt.grid()
plt.show()

plt.savefig('ndvimax_slope_aspect_twi_clay.png')

In [ ]:
optimal_cluster_num = 5
clhs_clust = KMeans(n_clusters=optimal_cluster_num, random_state=416).fit(clhs_inputs_st)
clhs_inputs['cluster'] = clhs_clust.labels_.tolist()

In [ ]:
strata = np.empty(greenest.shape)
strata[:] = np.nan
y = np.unravel_index(clhs_inputs.index, greenest.shape)[0]
x = np.unravel_index(clhs_inputs.index, greenest.shape)[1]
replacements = clhs_inputs['cluster'].values


for i in range(len(replacements)):
    strata[y[i],x[i]] = replacements[i]

plt.title('Strata')
plt.imshow(strata)

# Write Results to File


In [ ]:
utils.write_geotiff('ndvimax_twi_aspect_clay_7_classes.tif', strata, greenest_tf, greenest_crs)

In [ ]:
output_dir = os.path.join(working_dir, 'output')

ndvi_of = os.path.join(output_dir, 'max_ndvi_20200501_20210501.tif')
slope_of = os.path.join(output_dir, 'slope.tif')
aspect_of = os.path.join(output_dir, 'aspect.tif')
clay_of = os.path.join(output_dir, 'clay.tif')
twi_of = os.path.join(output_dir, 'twi.tif')
om_of = os.path.join(output_dir, 'om.tif')


utils.write_geotiff(ndvi_of, greenest, greenest_tf, greenest_crs)
utils.write_geotiff(slope_of, slope, elev_tf, elev_crs)
utils.write_geotiff(om_of, om, res_tf, greenest_crs)
utils.write_geotiff(clay_of, clay, greenest_tf, greenest_crs)
utils.write_geotiff(aspect_of, aspect, greenest_tf, greenest_crs)

Strata were smoothed over using the SAGA Majority filter - default settings

Random points were assigned based on % of total area

